## Dashboard Creation and Automated Reporting Instruction Manual

### Required Application Downloads

### Grab and Open Files

The cells inbetween the lines gets the data merges it moves the employees with multiple entries to the bottom of the table and then consolidates that specific multiple entry data and calculated the numbers off the merged data.

***

In [160]:
#Library Importation
import openpyxl
from openpyxl import load_workbook
import pandas as pd
import numpy as np

#Loading Necessary Workbooks
wb1 = load_workbook ("StaffPerformanceOverviewCR06-23-23 test copy.xlsx")
wb2 = load_workbook("StaffPerformanceOverviewNH06-23-23 test copy.xlsx")
ws1 = wb1.active
ws2 = wb2.active

#Appending Data To One Workbook
for row in ws2.iter_rows(min_row=2, values_only=True): 
    ws1.append(row)

wb1.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

#reload new workbook that was just saved
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")
ws = wb.active

#Remove Row to consolidate data
start_row = 12
num_rows_to_remove = 1
ws.delete_rows(start_row, num_rows_to_remove)

# Define the column letter you want to update (e.g., column A)
column_letter = 'E'

# Define the value to be replaced and its corresponding replacement
old = "NA"
new = "0"

# Get the column index from the column letter
column_index = ord(column_letter) - ord('A') + 1

# Loop through the cells in the selected column and replace the old value with the new value
for row in ws.iter_rows(min_row=3, min_col=column_index, max_col=column_index):
    cell = row[0]
    if cell.value == old:
        cell.value = new

from openpyxl.styles import NamedStyle

# Create named style for the whole number format
whole_number_style = NamedStyle(name="whole_number", number_format="0")

# Columns that need to be formatted as a whole number (example: columns A, B, and D)
columns_to_format = ['B', 'C', 'D']  # Replace with the letters of the columns you want to format

# Apply the named style to each cell in the selected columns
for column_letter in columns_to_format:
    for cell in ws[column_letter]:
        cell.style = whole_number_style

Decimal_Number_Style = NamedStyle(name="decimal", number_format="#,##0.00")

columns_to_format = ['E', 'F', 'G', 'H','I','J','K','L','M','N','O']

for column_letter in columns_to_format:
    for cell in ws[column_letter]:
        cell.style = Decimal_Number_Style


wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

In [161]:
from openpyxl import load_workbook

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the worksheet you want to update (assuming it's the first sheet)
ws = wb.active

# Find the last row in the worksheet
last_row = ws.max_row

# Create a dictionary to store the rows of employees with multiple entries
multiple_entries_rows = {}

# Iterate through the rows and collect the rows of employees with multiple entries
for row in range(3, last_row + 1):
    employee_id = ws.cell(row=row, column=1).value  # Assuming 'EmployeeID' is in column A
    if employee_id in multiple_entries_rows:
        multiple_entries_rows[employee_id].append([cell.value for cell in ws[row]])
    else:
        multiple_entries_rows[employee_id] = [[cell.value for cell in ws[row]]]

# Clear the existing data in the worksheet
ws.delete_rows(3, last_row)

# Write the rows of employees with single entries at the top of the worksheet
starting_row_for_multiple_entries = 11  # Adjust this variable for different reports
for employee_id, rows in multiple_entries_rows.items():
    if len(rows) == 1:
        ws.append(rows[0])  # Append the single row for employees with single entries

# Write the rows of employees with multiple entries at the bottom of the worksheet starting at the specified row
for employee_id, rows in multiple_entries_rows.items():
    if len(rows) > 1:
        for row_data in rows:
            ws.insert_rows(starting_row_for_multiple_entries, 1)  # Insert a row at the specified row for employees with multiple entries
            for col_index, cell_value in enumerate(row_data):
                ws.cell(row=starting_row_for_multiple_entries, column=col_index + 1, value=cell_value)
            starting_row_for_multiple_entries += 1  # Increment the starting row for the next employee

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")



In [162]:
import pandas as pd
from openpyxl import load_workbook

# Define custom functions for columns 14 and 15 based on columns 2, 3, and 4
def calculate_AvgbillN(row):
    # Net Average Bill
    return row['TotalN'] / row['Visits']

def calculate_AvgbillT(row):
    # Total Average Bill
    return row['TotalT'] / row['Visits']

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the worksheet you want to update (assuming it's the first sheet)
ws = wb.active

# Read the data from the worksheet into a Pandas DataFrame
data_range = ws['A11:O18']  # Assuming data is in columns A to K starting from row 2
data = [[cell.value for cell in row] for row in data_range]

# Create a DataFrame with all the data
df = pd.DataFrame(data, columns=['EmployeeID', 'Visits', 'New', 'RQs', 'Rat', 'ServiceN', 'ServiceT', 'SeriesN', 
                                  'SeriesT', 'ProductsN', 'ProductsT', 'TotalN', 'TotalT', 'AvgbillN', 'AvgbillT'])

# Get the union of EmployeeIDs
all_employee_ids = set(df['EmployeeID'])

# Create a DataFrame with all EmployeeIDs
df_all_employees = pd.DataFrame({'EmployeeID': list(all_employee_ids)})

# Merge the original data with the DataFrame containing all EmployeeIDs (outer merge)
merged_data = pd.merge(df_all_employees, df, on='EmployeeID', how='left')

# Fill NaN values with 0
merged_data.fillna(0, inplace=True)

# Convert 'Data4', 'Data13', and 'Data14' columns to numeric type (ignore errors)
merged_data['Rat'] = pd.to_numeric(merged_data['Rat'], errors='coerce')
merged_data['AvgbillN'] = pd.to_numeric(merged_data['AvgbillN'], errors='coerce')
merged_data['AvgbillT'] = pd.to_numeric(merged_data['AvgbillT'], errors='coerce')

# Consolidate data based on EmployeeID and calculate the sum for each of the columns 2 to 13
consolidated_data = merged_data.groupby('EmployeeID', as_index=False).agg({
    'Visits': 'sum',
    'New': 'sum',
    'RQs': 'sum',
    'Rat': 'sum',
    'ServiceN': 'sum',
    'ServiceT': 'sum',
    'SeriesN': 'sum',
    'SeriesT': 'sum',
    'ProductsN': 'sum',
    'ProductsT': 'sum',
    'TotalN': 'sum',
    'TotalT': 'sum',
    'AvgbillN': 'sum',
    'AvgbillT': 'sum'})

# Calculate the average for Data4, Data13 columns for each individual employee
consolidated_data['Rat'] = consolidated_data['Rat'] / 2

# Calculate the values for columns 14 and 15 using the custom functions based on other columns
consolidated_data['AvgbillN'] = consolidated_data.apply(calculate_AvgbillN, axis=1)
consolidated_data['AvgbillT'] = consolidated_data.apply(calculate_AvgbillT, axis=1)

# Determine the starting row for placing the consolidated data
start_row = 20  # Assuming you want to start placing the consolidated data in row 20

# Put the consolidated data back into the Excel worksheet
for row_index, row_data in enumerate(consolidated_data.values):
    for col_index, cell_value in enumerate(row_data):
        cell = ws.cell(row=start_row + row_index, column=col_index + 1)
        cell.value = cell_value

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

#The 3 cells above with code is exctly what transformations need to take place

***